In [1]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [2]:
import ampligraph
# Benchmark datasets are under ampligraph.datasets module
from ampligraph.datasets import load_fb15k_237
# load fb15k-237 dataset
dataset = load_fb15k_237()

In [3]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# create the model with transe scoring function
model = ScoringBasedEmbeddingModel(eta=1, 
                                     k=10,
                                     scoring_type='TransE')


# compile the model with loss and optimizer
model.compile(optimizer='adam', loss='multiclass_nll')

# Use this for checkpoints at regular intervals
checkpoint = tf.keras.callbacks.ModelCheckpoint('./chkpt1_transe', monitor='val_mrr', verbose=1, save_best_only=True, mode='min')

dataset = load_fb15k_237()

model.fit(dataset['train'],
             batch_size=10000,
             epochs=10,
             validation_freq=2,
             validation_batch_size=100,
             validation_data = dataset['valid'][::100],
         callbacks=[checkpoint])     # Pass the callback to the fit function


Epoch 1/10
29/29 [==============================] - 2s 75ms/step - loss: 6698.2188
Epoch 2/10
29/29 [==============================] - 2s 71ms/step - loss: 6648.8862 - val_mrr: 0.0672 - val_mr: 2604.4943 - val_hits@1: 0.0000e+00 - val_hits@10: 0.1847

Epoch 2: val_mrr improved from inf to 0.06722, saving model to ./chkpt1_transe
WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.
Epoch 3/10
29/29 [==============================] - 1s 32ms/step - loss: 6590.2842
Epoch 4/10
29/29 [==============================] - 1s 43ms/step - loss: 6517.4517 - val_mrr: 0.0918 - val_mr: 1316.6335 - val_hits@1: 0.0000e+00 - val_hits@10: 0.2528

Epoch 4: val_mrr did not improve from 0.06722
Epoch 5/10
29/29 [==============================] - 1s 32ms/step - loss: 6431.8696
Epoch 6/10
29/29 [==============================] - 1s 45ms/step - loss: 6338.9854 - val_mrr: 0.0873 - val_mr: 935.4460 - val_hits@1: 0.0000e+

In [4]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], # test set
                       batch_size=100, # evaluation batch size
                       corrupt_side='s,o', 
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 85s 410ms/step
MR: 683.0425188374596
MRR: 0.15564075295325883
hits@1: 0.10260299442215481
hits@10: 0.258929445151189


In [5]:
from ampligraph.utils import save_model
# explictly save the model
save_model(model, 'saved_model_transE')


The path saved_model_transE already exists. This save operation will overwrite the model                 at the specified path.
WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.


In [6]:
from ampligraph.utils import restore_model

# restore saved models or checkpoints
model = restore_model('saved_model_transE')

In [7]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], # test set
                       batch_size=100, # evaluation batch size
                       corrupt_side='s,o', 
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 85s 413ms/step
MR: 683.0425188374596
MRR: 0.15564075295325883
hits@1: 0.10260299442215481
hits@10: 0.258929445151189
